In [156]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException

import pandas as pd
import time
#brew install geckodrivee

In [611]:
def error_parse(error):
    if "trustee" in error:
        return "CUTrustee"
    return error

def email_chars(email):
    if email:
        if '•' in email:
            sublen = len(email.split("@")[0])-2
            return email.split("@")[0][:2]+"?"*sublen+email.split("@")[1]
    return email

In [666]:
driver = webdriver.Firefox()

In [667]:
###to shut things down:
# elem.clear()
# elem.send_keys("pycon")
# elem.send_keys(Keys.RETURN)
# assert "No results found." not in driver.page_source
# driver.close()

###getting other things done:
# dir(By)
# print(driver.title)
# print(dir(driver))
# 
# ids = elem.find_elements_by_xpath('//*[@id]')

In [689]:
driver.get("https://tools.alumni.columbia.edu/uni/SelectUser.aspx")

In [690]:
elem = driver.find_element_by_id("container")

In [691]:
firstname = elem.find_elements_by_id("ctl00_rightcol_ph_first_tb")

In [692]:
lastname = elem.find_elements_by_id("ctl00_rightcol_ph_last_tb")
searchbutton=elem.find_elements_by_id("ctl00_rightcol_ph_Button2")
school=elem.find_elements_by_id("ctl00_rightcol_ph_school_ddl_Input")

In [ ]:
alumni = []
hiccup_letters = []
hiccup_emails = []
for class_year in range(1970,1989):
    for firstchar in "abcdefghijklmnopqrstuvwxyz":
        for lastchar in "abcdefghijklmnopqrstuvwxyz":
            try:
                school=elem.find_elements_by_id("ctl00_rightcol_ph_school_ddl_Input")
                school[0].click()
                college=driver.find_element_by_id("ctl00_rightcol_ph_school_ddl_c6").click()
                year = elem.find_element_by_id("ctl00_rightcol_ph_year_ddl")
                year.click()
                year.send_keys(str(class_year))

                firstname[0].send_keys(firstchar)
                lastname[0].send_keys(lastchar)
                searchbutton[0].send_keys(Keys.ENTER)
                #time.sleep(5)
                try:
                    table = driver.find_element_by_id("ctl00_rightcol_ph_result_grid")
                except NoSuchElementException:
                    time.sleep(5)
                    table = driver.find_element_by_id("ctl00_rightcol_ph_result_grid")

                people = table.find_elements(By.TAG_NAME, "tr")
                if len(people)>1:

                    for person in people[1:]:

                        info = person.text
                        name = info.split("'")[0].rstrip()
                        other = "".join(info.split("'")[1:])
                        alumni.append([name,other,int(str(class_year)[2:]),"CC"])

                    for record in alumni[(-len(people)+1):]:
                        try:
                            link = driver.find_element_by_link_text(record[0])
                            link.click()
                        except:
                            driver.back()
                            link = driver.find_element_by_link_text(record[0])
                            link.click()
                        try:
                            time.sleep(2)
                            email_table=driver.find_element_by_id("ctl00_leftcol_ph_emails_rbl")
                            email_list = email_table.find_elements(By.TAG_NAME, "tr")

                            for emails in email_list:
                                time.sleep(1)
                                record.append(email_chars(emails.text))
                                if len(record[-1])<2:
                                    time.sleep(2)
                                    record[-1]=email_chars(emails.text)
                                    if len(record[-1])<2:
                                        raise Exception()
                                        print("Check hiccup emails for email that isn't working for "+str([person.text,emails.text]))
                                        hiccup_emails.append([person.text,emails.text])

                            driver.back()

                        except NoSuchElementException:
                            #record.append(email)

                            try:
                                error_message = driver.find_element_by_id("ctl00_rightcol_ph_ErrorPnl")
                                if type(error_message)!=str:
                                    error_message = error_message.text
                                record.append(error_parse(error_message))
                            except NoSuchElementException:
                                error_message = "other error"
                                record.append(error_message)
                                driver.back()
                                continue
                                
            except Exception:
                print("hiccup on:")
                print(firstchar)
                print(lastchar)
                hiccup_letters.append([firstchar,lastchar])
                break
            
            driver.get("https://tools.alumni.columbia.edu/uni/SelectUser.aspx")
            attempts = 0
            while attempts < 10:
                try:
                    driver.get("https://tools.alumni.columbia.edu/uni/SelectUser.aspx")
                    break
                except:
                    driver.refresh()
                    time.wait(5)
                    attempts = attempts + 1
                    continue
                
            if attempts == 10:
                print("SOMETHING WRONG: 10 ATTEMPTS & NOTHING FIXED")
                
            elem = driver.find_element_by_id("container")
            firstname = elem.find_elements_by_id("ctl00_rightcol_ph_first_tb")
            lastname = elem.find_elements_by_id("ctl00_rightcol_ph_last_tb")
            searchbutton=elem.find_elements_by_id("ctl00_rightcol_ph_Button2")
            school=elem.find_elements_by_id("ctl00_rightcol_ph_school_ddl_Input")
            
    pd.DataFrame(alumni).to_csv("CC_"+str(class_year)+"_alumni.csv")
            

In [676]:
alumni[0][4][3] == '•'

True

In [684]:
if None:
    print("yes")

In [678]:
email = alumni[0][4]

In [679]:
email.split("@")

['al••••', 'nygasp.org']

In [681]:
email_chars(email)

'al????nygasp.org'

In [590]:
table = driver.find_element_by_id("ctl00_rightcol_ph_result_grid")
people = table.find_elements(By.TAG_NAME, "tr")


In [665]:
alumni

[['Albert H. Bergeret',
  '70CC ahb30',
  70,
  'CC',
  'al••••@nygasp.org',
  'in••@nygasp.org'],
 ['Arthur D. Bramble', '70CC adb56', 70, 'CC', 'other error'],
 ['Andre E. Broussard',
  '70CC aeb58',
  70,
  'CC',
  'ae•••@caa.columbia.edu',
  'an•••••••••@yahoo.com',
  'an•••••••••••••@nychhc.org',
  'an••••••@hotmail.com'],
 ['Allen E. Buchanan', '70CC aeb61', 70, 'CC', 'other error'],
 ['Alan I. Charney', '70CC aic16', 70, 'CC', 'other error'],
 ['Anthony P. Ciambrone', '70CC apc25', 70, 'CC', 'other error'],
 ['Anthony J. Cicconi', '70CC ajc62', 70, 'CC', 'za••••••••@aol.com'],
 ['Ates Dagli',
  '70CC 80GSAS cad54',
  70,
  'CC',
  'ca•••@caa.columbia.edu',
  'ca•••••@caa.columbia.edu',
  'at••••••••@us.ibm.com',
  'at••@spss.com'],
 ['Angus K. Davis', '70CC akd25', 70, 'CC', 'ao•••••••@gmail.com'],
 ['A. James Epstein', '70CC aje12', 70, 'CC', 'other error'],
 ['Arthur J. Epstein', '70CC aje12', 70, 'CC', 'other error']]

In [541]:
email_table=driver.find_element_by_id("ctl00_leftcol_ph_emails_rbl")
email_list = email_table.find_elements(By.TAG_NAME, "tr")

for emails in email_list:

    x.append(emails.text)

In [542]:
x

[None, None, 'al••••@gmail.com', 'al••••@aol.com']

In [587]:
alumni

[['Albert H. Bergeret',
  "'70CC",
  'ahb30',
  'al••••@nygasp.org',
  'in••@nygasp.org'],
 ['Arthur D. Bramble', "'70CC", 'adb56', 'other error'],
 ['Andre E. Broussard',
  "'70CC",
  'aeb58',
  'ae•••@caa.columbia.edu',
  'an•••••••••@yahoo.com',
  'an•••••••••••••@nychhc.org',
  'an••••••@hotmail.com'],
 ['Allen E. Buchanan', "'70CC", 'aeb61', 'other error'],
 ['Alan I. Charney', "'70CC", 'aic16', 'other error'],
 ['Anthony P. Ciambrone', "'70CC", 'apc25', 'other error'],
 ['Anthony J. Cicconi', "'70CC", 'ajc62', 'za••••••••@aol.com'],
 ['Ates Dagli',
  "'80GSAS",
  'cad54',
  'ca•••@caa.columbia.edu',
  'ca•••••@caa.columbia.edu',
  'at••••••••@us.ibm.com',
  'at••@spss.com'],
 ['Angus K. Davis', "'70CC", 'akd25', 'ao•••••••@gmail.com'],
 ['A. James Epstein', "'70CC", 'aje12', 'other error'],
 ['Arthur J. Epstein', "'70CC", 'aje12', 'other error'],
 ['Andrew N. Hartman', "'70CC", 'anh9', 'other error'],
 ['Andy Kiorpes', "'70CC", 'alk59', 'an••@kiorpes.net', 'a.•••••••@att.net'],
 

In [61]:
##getting the components on the page:
ids = elem.find_elements_by_xpath('//*[@id]')
for ii in ids:
    #print ii.tag_name
    print(ii.get_attribute('id'))

aspnetForm
__EVENTTARGET
__EVENTARGUMENT
__VIEWSTATE
__VIEWSTATEGENERATOR
__VIEWSTATEENCRYPTED
__EVENTVALIDATION
container
page
page-inner
header
branding
brand-wrapper
brand-wrapper-inner
branded-site-name
site-name
block-cu-admin1-mainnavigation
block-cu-admin1-mainnavigation-menu
site-nav
site-nav-second
mobile-nav
mobile-nav-container
sidebar-left
sidebar-left-inner
content-area
sidebar-left
sidebar-left-inner
ctl00_rightcol_ph_RadTabStrip1
ctl00_rightcol_ph_RadTabStrip1StyleSheetHolder
ctl00_rightcol_ph_RadTabStrip1_Tab1
ctl00_rightcol_ph_RadTabStrip1_Tab2
ctl00_rightcol_ph_RadTabStrip1_Hidden
ctl00_rightcol_ph_RadMultiPage1
ctl00_rightcol_ph_name_view
ctl00_rightcol_ph_byname
ctl00_rightcol_ph_first_tb
ctl00_rightcol_ph_last_tb
ctl00_rightcol_ph_school_ddl_wrapper
ctl00_rightcol_ph_school_ddl
ctl00_rightcol_ph_school_ddl_Input
ctl00_rightcol_ph_school_ddl_Image
ctl00_rightcol_ph_school_ddl_DropDownPlaceholder
ctl00_rightcol_ph_school_ddl_DropDown
ctl00_rightcol_ph_school_ddl_c0
c